## Etapa atual
- **Validação da ideia**: criação de uma versão em excel para comparação de preços mercados fisicos.  
- **Criação de base inicial**: coletar dados de algumas notas fiscais ou produtos.  
- **Objetivo desta etapa**: ter **uma base inicial de preços** para análises e comparativos.

##  Estrutura para etapa atual
1. Criar **script para raspar dados das notas fiscais**:  
   - Extrair: produto, preço, mercado, data.  
2. Alimentar **planilha ou banco de dados simples** (CSV / Google Sheets / SQLite).  
3. Fazer **comparativo de preços iniciais** usando esses dados.  
4. Validar se o MVP manual fornece resultados confiáveis antes de automatizar scraping de sites ou QR Codes.


In [30]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [ ]:
# SCRIPT PARA RASPAGEM DE DADOS


links = [
    "https://www.nfce.fazenda.sp.gov.br/NFCeConsultaPublica/Paginas/ConsultaQRCode.aspx?p=35250944914620000101651070000018431107018733|2|1|1|6D1F127B092ACFC5A9130DD51D27B2DA25F4B2A7",
    "https://www.nfce.fazenda.sp.gov.br/NFCeConsultaPublica/Paginas/ConsultaQRCode.aspx?p=35250944914620000101651070000018191107018487|2|1|1|3BDBE9933615AFAE4F6D1CDC6305D3F8C59736D4",
    "https://www.nfce.fazenda.sp.gov.br/NFCeConsultaPublica/Paginas/ConsultaQRCode.aspx?p=35250944914620000101651040000018481104018810|2|1|1|B30157F82C9F36B633C9A0E2104CDE45F8AD592A",
    "https://www.nfce.fazenda.sp.gov.br/NFCeConsultaPublica/Paginas/ConsultaQRCode.aspx?p=35250944914620000101651040000018431104018768|2|1|1|DDE68EEA0BBADF62BBF8CA252FF7C578F4AC4DB5",
    "https://www.nfce.fazenda.sp.gov.br/NFCeConsultaPublica/Paginas/ConsultaQRCode.aspx?p=35250944914620000101651020000029821102030260|2|1|1|7CEABA67901306C7669AF902F102EC944F6cc82413",
    "https://www.nfce.fazenda.sp.gov.br/NFCeConsultaPublica/Paginas/ConsultaQRCode.aspx?p=35250944914620000101651040000018621104018968|2|1|1|A71446FE9BE1660604CAD5B7C69239BF7D6E214D"
]
def cadastro_nome_produto(dados):
    tit_produto = dados.find_all('span', class_='txtTit')
    nomes_produtos = [p.get_text(strip=True) for p in tit_produto]
    return nomes_produtos

def cadastro_quantidade_comprada(dados):
    quantidade = dados.find_all('span', class_='Rqtd')
    quantidade_comprada = [p.get_text(strip=True).replace("Qtde.:", "") for p in quantidade]
    return quantidade_comprada

def cadastro_unidade_medida(dados):
    unidade = dados.find_all('span', class_='RUN')
    unidade_produto = [p.get_text(strip=True).replace("UN:", "") for p in unidade]
    return unidade_produto

def cadastro_valor_unitario(dados):
    valor_unitario = dados.find_all('span', class_='RvlUnit')
    vl_unitario = [
        p.get_text(strip=True).replace("Vl. Unit.:", "").replace(",", ".")
        for p in valor_unitario
    ]
    return vl_unitario

def cadastro_valor_total(dados):
    valor_total = dados.find_all('td', class_='txtTit noWrap')
    vl_total = [
        p.get_text(strip=True).replace("Vl. Total", "").replace(",", ".")
        for p in valor_total
    ]
    return vl_total

def cadastro_nome_mercado(dados):
    nome_empr = dados.find_all('div', class_='txtTopo')
    nome_empresa = [p.get_text(strip=True) for p in nome_empr]
    return nome_empresa

def cadastro_cnpj_empresa(dados): 
    documento = dados.find_all('div', class_='text')
    cnpjs = []
    for p in documento:
        texto = p.get_text(strip=True)
        if "CNPJ:" in texto:  # garante que só pega o bloco do CNPJ
            cnpjs.append(texto.replace("CNPJ:", "").strip())
    return cnpjs


def cadastro_data_emissao(dados):
    tag = dados.find("strong", string=lambda s: s and "Emissão" in s)
    if tag:
        data_texto = tag.next_sibling.strip()
        data_emissao = data_texto.split("-")[0].strip()
        return data_emissao
    return None


def cria_dataframe(dados):
    # usa as funções que você já criou
    cnpj = cadastro_cnpj_empresa(dados)
    nome_empresa = cadastro_nome_mercado(dados)
    nomes_produtos = cadastro_nome_produto(dados)
    quantidade_comprada = cadastro_quantidade_comprada(dados)
    unidade_produto = cadastro_unidade_medida(dados)
    vl_unitario = cadastro_valor_unitario(dados)
    vl_total = cadastro_valor_total(dados)
    data_emissao = cadastro_data_emissao(dados)

    # garante que os dados "fixos" repitam para cada produto
    cnpj = [cnpj[0]] * len(nomes_produtos)
    nome_empresa = [nome_empresa[0]] * len(nomes_produtos)
    data_emissao = [data_emissao] * len(nomes_produtos)

    # monta dicionário
    dados_dicionario = {
        'CNPJ': cnpj,
        'Empresa': nome_empresa,
        'Data Emissão': data_emissao,
        'Produto': nomes_produtos,
        'Qtde': quantidade_comprada,
        'UN': unidade_produto,
        'Vl. Unit.': vl_unitario,
        'Vl. Total': vl_total
    }

    # cria dataframe
    df = pd.DataFrame(dados_dicionario)

    # converte valores numéricos
    df['Qtde'] = df['Qtde'].str.replace(",", ".").astype(float)
    df['Vl. Unit.'] = df['Vl. Unit.'].astype(float)
    df['Vl. Total'] = df['Vl. Total'].astype(float)

    return df

dfs = []

for url in links:
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            df_nfe = cria_dataframe(soup)
            dfs.append(df_nfe)
        else:
            print(f"Erro ao acessar {url}: {response.status_code}")
    except Exception as e:
        print(f"Erro ao processar {url}: {e}")

# Concatenar todas as notas em um único DataFrame
df_final = pd.concat(dfs, ignore_index=True)

# Salvar em CSV
df_final.to_excel("notas_raspadas.xlsx", index=False)

In [32]:
df_final

,CNPJ,Empresa,Data Emissão,Produto,Qtde,UN,Vl. Unit.,Vl. Total
0,44.914.620/0001-01,MERCADO JARDIM ZAIRA LTDA,26/09/2025 19:46:00,CALDO SEARA 300GR VERDE CALABRESA,1.000,UN0001,17.990000,17.99
1,44.914.620/0001-01,MERCADO JARDIM ZAIRA LTDA,26/09/2025 19:46:00,CALDO SEARA 300GR MANDIOQUINHA FRANGO,1.000,UN0001,17.990000,17.99
2,44.914.620/0001-01,MERCADO JARDIM ZAIRA LTDA,26/09/2025 19:46:00,SEARA NETFLIX 245GR MINI HOT DOG COREANO,1.000,UN0001,18.990000,18.99
3,44.914.620/0001-01,MERCADO JARDIM ZAIRA LTDA,26/09/2025 19:46:00,CHOCOLATE SNICKERS 42GR DARK,1.000,UN0001,5.990000,5.99
4,44.914.620/0001-01,MERCADO JARDIM ZAIRA LTDA,26/09/2025 19:46:00,CHOCOLATE SNICKERS 45GR,1.000,UN0001,5.990000,5.99
5,44.914.620/0001-01,MERCADO JARDIM ZAIRA LTDA,26/09/2025 19:46:00,REFRIG COCA COLA PET 2L,1.000,UN0001,10.990000,10.99
6,44.914.620/0001-01,MERCADO JARDIM ZAIRA LTDA,26/09/2025 19:07:00,BATATA PALHA KISABOR 100GR TRADICIONAL,1.000,UN0001,6.990000,6.99
7,44.914.620/0001-01,MERCADO JARDIM ZAIRA LTDA,26/09/2025 19:07:00,REFRIG CRUZEIRO PET 2L GUARANA,1.000,UN0001,5.990000,5.99
8,44.914.620/0001-01,MERCADO JARDIM ZAIRA LTDA,26/09/2025 19:07:00,PAO KEROKERO 600GR HOT DOG,1.000,UN0001,10.990000,10.99
9,44.914.620/0001-01,MERCADO JARDIM ZAIRA LTDA,26/09/2025 19:07:00,BONOBON BEIJINHO 15G,2.000,UN0001,1.490000,2.98
